# Terceira lista de visão computacional
# Lucas Heron Santos Anchieta
# Ruan Tenório de Melo

## Importações

In [321]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


In [322]:
def load_image(url, code = cv.IMREAD_COLOR):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv.imdecode(image, code)
  return image

## 1. Escolha uma das metodologias que você implementou na segunda lista para gerar correspondências entre um par de imagens. Aplique-a em 5 pares de imagens (com sobreposição) para calcular suas homografias, e aplique-as para gerar panoramas entre os pares de imagens (um panorama por par).
## Obs.: nessa questão, não é permitido usar a API de alto nível Stitcher.
## Dica: use a função warpPerspective da OpenCV.

In [323]:
def flan_matcher(des1, des2, index_params, search_params):
  # Create FLANN matcher
  flann = cv.FlannBasedMatcher(index_params, search_params)

  # Match descriptors
  matches = flann.knnMatch(des1, des2, k=2)

  

  good_matches = []
  ratio_threshold = 0.9
  for m, n in matches:
      if m.distance < ratio_threshold * n.distance:
          good_matches.append(m)
  good_matches = sorted(good_matches, key=lambda x: x.distance)

  return good_matches

In [324]:
def draw_matches_mpl(img1, keypoints1, img2, keypoints2, matches):
    """Draw matches using Matplotlib."""

    # Combine images horizontally
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    combined_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    combined_img[:h1, :w1] = img1
    combined_img[:h2, w1:] = img2

    plt.figure(figsize=(12, 6))
    plt.imshow(combined_img)
    plt.axis('off')
  
    # Draw lines connecting the matches
    offset = w1
    for match in matches:
        pt1 = keypoints1[match.queryIdx].pt
        pt2 = keypoints2[match.trainIdx].pt
        x1, y1 = int(pt1[0]), int(pt1[1])
        x2, y2 = int(pt2[0] + offset), int(pt2[1])
        plt.plot([x1, x2], [y1, y2], 'r-', linewidth=0.5)
        plt.plot(x1, y1, 'bo', markersize=3)  # Mark keypoints
        plt.plot(x2, y2, 'bo', markersize=3)

    return plt

In [325]:
def descriptor_akaze(img):
  akaze = cv.akaze_create()
  # Detect keypoints and compute descriptors
  kp, des = akaze.detectAndCompute(img, None)

  return kp, des


In [ ]:
def get_homograph(kp1, kp2, matches): 
  src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1, 1, 2)
  dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1, 1, 2)
  H, mask =  cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

  return H, mask


In [353]:
def compute_panorama(img1, img2):
    img1_gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    img2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    akaze = cv.AKAZE_create()

    # Encontrar keypoints e descritores
    kp1, des1 = akaze.detectAndCompute(img1_gray, None)
    kp2, des2 = akaze.detectAndCompute(img2_gray, None)

    
    des1 = des1.astype(np.float32)
    des2 = des2.astype(np.float32)

    good_matches = flan_matcher(des1, des2, index_params, search_params)

    # Find homography
    M, _ = get_homograph(kp1, kp2, good_matches)

    if M is None:
        return np.hstack((img1, img2))
    img1_height, img1_width = img1.shape[:2]
    img2_height, img2_width = img2.shape[:2]
    # Get the corners of img1
    corners1 = np.array([[0, 0], [img1_width, 0], [img1_width, img1_height], [0, img1_height]], dtype=np.float32)
    # Transform corners of img1 to img2's perspective
    corners1_transformed = cv.perspectiveTransform(corners1.reshape(-1, 1, 2), M)
    # Get the corners of img2
    corners2 = np.array([[0, 0], [img2_width, 0], [img2_width, img2_height], [0, img2_height]], dtype=np.float32)

    # Combine corners
    all_corners = np.concatenate((corners1_transformed, corners2.reshape(-1, 1, 2)), axis=0)
    # Get the bounding box of all corners
    [x_min, y_min] = np.int32(all_corners.min(axis=0).ravel())
    [x_max, y_max] = np.int32(all_corners.max(axis=0).ravel())
    # Adjust the translation matrix to include the bounding box

    # Create a new image with the size of the bounding box
    panorama_width = x_max - x_min
    panorama_height = y_max - y_min
    panorama = np.zeros((panorama_height, panorama_width, 3), dtype=np.uint8)
    translation_dist = [-x_min, -y_min]
      # Create a translation matrix to shift the panorama to the correct position
    translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
    # Warp img1 to the panorama
    panorama = cv.warpPerspective(img1, translation_matrix @ M, (panorama_width, panorama_height))
    
    # Combine the two warped images
    # panorama = cv.addWeighted(panorama1, 0.5, panorama2, 0.5, 0)
    # Return the panorama
    panorama[translation_dist[1]:img2_height + translation_dist[1], translation_dist[0]:img2_width + translation_dist[0]] = img2
    return panorama
      

In [328]:
# orb.cv.ORB_create()

In [ ]:
img1 = cv.imread('./img/sofa1.jpg')
img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)

# h, w = img1.shape[:2]

# img1 = cv.resize(img1, (w//8, h//8), interpolation=cv.INTER_CUBIC)

plt.imshow(img1)

In [ ]:
img2 = cv.imread('./img/sofa2.jpg')
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)

plt.imshow(img2)

In [357]:
plt.imshow(compute_panorama(img1, img2))
plt.axis('off')

In [332]:
# M, mask = get_homograph(kp1, kp2, flan_matcher)

# h1, w1 = img1.shape[:2]
# h2, w2 = img2.shape[:2]

# corners_img1 = np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2)
# warped_corners_img1 = cv.perspectiveTransform(corners_img1, M)

# all_corners = np.concatenate((warped_corners_img1, np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)), axis=0)

# [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
# [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)

# translation_dist = [-xmin, -ymin]
# H_translation = np.array([[1, 0, translation_dist[0]],
#                           [0, 1, translation_dist[1]],
#                           [0, 0, 1]])

# panorama = cv.warpPerspective(img1, H_translation @ M, (xmax - xmin, ymax - ymin))

# panorama[translation_dist[1]:h2 + translation_dist[1], translation_dist[0]:w2 + translation_dist[0]] = img2

## 2. Repita a questão anterior com 5 trios de imagens (com sobreposição 2 a 2), alinhando as imagens no plano da primeira imagem. Repita o mesmo alinhando no plano da segunda imagem, e da terceira imagem. Note que aqui será necessário compor as transformações de homografia em alguns casos, ou calcular inversas.

In [382]:
img1 = cv.imread('./img/rua1_resized.jpg')
img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
img1 = cv.flip(img1, -1)

img2 = cv.imread('./img/rua2_resized.jpg')
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)
img2 = cv.flip(img2, -1)

img3 = cv.imread('./img/rua3_resized.jpg')
img3 = cv.cvtColor(img3, cv.COLOR_BGR2RGB)
img3 = cv.flip(img3, -1)

panorama1 = compute_panorama(img1, img2)
plt.imshow(cv.flip(panorama1, -1))
plt.title('Panorama Part 1 (2 images)') 
plt.axis('off')
plt.show()

panorama2 = compute_panorama(img3, panorama1)
panorama2 = cv.flip(panorama2, -1)
plt.title('Panorama Part 2 (3 images)')
plt.axis('off')
plt.imshow(panorama2)
plt.show()

## 3. Considere a imagem soccer.jpg (./img/soccer.jpg) em anexo no Google Classroom. Considere que o campo da imagem tenha as dimensões dadas pela figura (./img/q3_anexo.png)
## Gere manualmente correspondências entre a imagem e um mapa 2d com dimensões dadas pela figura. Calcule a homografia resultante e aplique na imagem original. Exiba o resultado. Dica: leia este tutorial (https://medium.com/acmvit/how-to-project-an-image-in-perspective-view-of-a-background-image-opencv-python-d101bdf966bc)

In [333]:
img = cv.imread("./img/soccer.jpg")
field = cv.imread("./img/q3_anexo.png")

h_field, w_field = field.shape[:2]
resized_img = cv.resize(img, (w_field, h_field))

positions = []
count = 0
pts1 = []
pts2 = []
current_image = None

In [334]:
def draw_circle(event, x, y, flags, param):
    global positions, count, current_image

    if event == cv.EVENT_LBUTTONUP:
        if count >= 4:
            return
        cv.circle(current_image, (x,y), 2, (255, 0, 0), -1)

        positions.append((x,y))
        
        count += 1
        if count > 1:
            cv.line(current_image, positions[-2], positions[-1], 
                   (255, 0, 255), 2)

In [335]:
current_image = field.copy()
positions = []
count = 0
cv.namedWindow('image')
cv.setMouseCallback('image', draw_circle)

while(True):
    cv.imshow('image', current_image)
    k = cv.waitKey(20) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()

pts1 = np.float32(positions)
print("Collected points:\n", pts1)

In [336]:
current_image = resized_img.copy()
positions = []
count = 0
cv.namedWindow('image')
cv.setMouseCallback('image', draw_circle)

while(True):
    cv.imshow('image', current_image)
    k = cv.waitKey(20) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()

pts2 = np.float32(positions)
print("Collected points:\n", pts2)

In [337]:
h, _ = cv.findHomography(pts2, pts1, cv.RANSAC, 5.0)
print(f"Matriz de homografia: \n", h)

In [ ]:
mask_field = np.zeros((h_field, w_field), dtype=np.uint8)
cv.fillConvexPoly(mask_field, pts1.astype(int), 255)

In [ ]:
mask_img = np.zeros((h_field, w_field), dtype=np.uint8)
cv.fillConvexPoly(mask_img, pts2.astype(int), 255)

In [ ]:
warped_img = cv.warpPerspective(resized_img, h, (w_field, h_field))


In [ ]:
field_bg = cv.bitwise_and(field, field, mask=cv.bitwise_not(mask_field))
img_fg = cv.bitwise_and(warped_img, warped_img, mask=mask_field)

In [ ]:
result = cv.add(field_bg, img_fg)

# Mostra o resultado
while(True):
    cv.imshow('image', result)
    k = cv.waitKey(20) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()

## 4. Leia o seguinte tutorial de calibração de câmera: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
## Você vai precisar de um tabuleiro de xadrez (pode imprimir numa folha A4, e colar num papelão ou emplastificar para a geometria ficar fixa). Meça as dimensões do seu tabuleiro para calibrar a câmera, considerando que o tabuleiro sempre está no plano z = 0, e que o canto inferior esquerdo do tabuleiro é a origem (0, 0, 0). Após calibrar a câmera, vamos incluir um objeto virtual na imagem. Considere a seguinte equação paramétrica do círculo centrado no ponto (1.5W, 1.5H, 0), com raio r = 0.5W e contido no plano z = 0, onde H e W são a as medidas da altura e largura do tabuleiro:

## p(θ) = (r cos θ + 1.5W, r sen θ + 1.5H, 0).

## Se assegure de que o círculo apareça na imagem, de acordo com a posição do xadrez na imagem. Para desenhar o círculo, varie o valor do angulo θ entre 0 e 2π para amostrar alguns pontos, e projete-os na imagem. Repita isso 3 vezes, variando o ângulo entre o vetor normal do tabuleiro e o eixo principal da câmera.